In [1]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12
import sys
orig_stdout = sys.stdout
NoOfClasses = 2
dataType = 'discrete'
sys.stdout = orig_stdout

In [2]:
import numpy as np
sys.path.append("../")
import model_only

gold_labels_dev = np.load("data/true_labels_dev.npy")
gold_labels_test = np.load("data/true_labels_test.npy")

unique, counts = np.unique(gold_labels_dev, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(gold_labels_test, return_counts=True)
print(dict(zip(unique, counts)))

print(len(gold_labels_dev),len(gold_labels_test))

{0: 2625, 1: 189}
{0: 2484, 1: 218}
2814 2702


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
    print(precision_recall_fscore_support(gold_labels_test,pl,average='binary'))
    print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
#     return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [4]:
if dataType == 'merged':
    LF_l = [
        1,1,1,1,1,-1,1,-1,-1,-1
    ]
    dscr = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
    def merge(a,b):
        c = []
        for i in range(len(a)):
            ci = []
            ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
            ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
            ci.append(ci_l)
            ci.append(ci_s)
            c.append(ci)
        return c
    import numpy as np
    dev_L_S_s = np.load("data/dev_L_S_smooth.npy")
    test_L_S_s = np.load("data/test_L_S_smooth.npy")
    train_L_S_s = np.load("data/train_L_S_smooth.npy")

    dev_L_S_d = np.load("data/dev_L_S_discrete.npy")
    test_L_S_d = np.load("data/test_L_S_discrete.npy")
    train_L_S_d = np.load("data/train_L_S_discrete.npy")

    dev_L_S = np.array(merge(dev_L_S_d,dev_L_S_s))
    train_L_S = np.array(merge(train_L_S_d,train_L_S_s))
    test_L_S = np.array(merge(test_L_S_d,test_L_S_s))

    LF_l = LF_l + LF_l
    NoOfLFs= len(LF_l)

    print(len(LF_l))

    print(dev_L_S.shape, test_L_S.shape, train_L_S.shape)
    print(train_L_S[100])

In [5]:
if dataType == 'discrete':
    LF_l = [
        1,1,1,1,1,-1,1,-1,-1,-1
    ]
    NoOfLFs= len(LF_l)
    dscr = np.ones(10)

    dev_L_S = np.load("data/dev_L_S_discrete.npy")
    test_L_S = np.load("data/test_L_S_discrete.npy")
    train_L_S = np.load("data/train_L_S_discrete.npy")

    print(len(gold_labels_dev),len(gold_labels_test))
    print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)
    print(train_L_S[100])

2814 2702
(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)
[[ 0.   0.   0.   0.   0.   0.   0.  -1.  -1.   0. ]
 [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  1.   1.   0.5]]


In [6]:
if dataType == 'cont':
    LF_l = [
        1,1,1,1,1,-1,1,-1,-1,-1
    ]
    dscr = np.array([1,1,0,0,1,1,0,0,0,0])
    NoOfLFs= len(LF_l)

    dev_L_S = np.load("data/dev_L_S_smooth.npy")
    test_L_S = np.load("data/test_L_S_smooth.npy")
    train_L_S = np.load("data/train_L_S_smooth.npy")

    print(len(gold_labels_dev),len(gold_labels_test))
    print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)
    print(train_L_S[0:10])

In [7]:

discr = []
for j in range(len(LF_l)):
    temp = np.array(train_L_S[:,0,j]*np.array(train_L_S[:,1,j]))
    unique, counts = np.unique(temp, return_counts=True)
    temp_d = dict(zip(unique, counts))
    print(j, '\t', len(temp_d), temp_d if len(temp_d)==2 else '')
    discr.append(int(not(len(temp_d)-2)))
sys.stdout.write('[')
for x in range(len(discr)-1):
    sys.stdout.write(str(discr[x]) + ', ')
sys.stdout.write(str(discr[-1])+']')

print(len(LF_l))

sys.stdout = orig_stdout
print('test')

0 	 2 {0.0: 22243, 1.0: 33}
1 	 2 {0.0: 22096, 1.0: 180}
2 	 2 {0.0: 19945, 1.0: 2331}
3 	 2 {0.0: 20538, 1.0: 1738}
4 	 2 {0.0: 21904, 1.0: 372}
5 	 2 {-1.0: 13366, 0.0: 8910}
6 	 2 {0.0: 22261, 1.0: 15}
7 	 2 {-1.0: 2323, 0.0: 19953}
8 	 2 {-1.0: 1634, 0.0: 20642}
9 	 2 {-1.0: 208, 0.0: 22068}
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]10
test


In [ ]:
#added new model
def train2(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True, \
           r1=1.0, r2=1.0, a_pr=None, n_pr=None, user_a = None, isdiscrete = None, alpha_max = None, r_pr=None):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()

    seed = 12
    
    LF_k = []
    if(len(pcl)==2):
        for i in range(len(LF_l)):
            LF_k.append(int((LF_l[i]+1)/2))
    with tf.Graph().as_default():
        xtra = tf.Variable(0, trainable=False)
        gammaR = tf.constant(0, dtype=tf.float64)

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

        iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()

        thetas,alphas = model_only.allocate_params(NoOfLFs, 2, penalty, th, af)
        
        LF_label = tf.convert_to_tensor(LF_k, dtype=tf.int32)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11, 44, 42, 41]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12,15]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        l,s =  tf.unstack(next_element,axis=1)

        if(smooth and penalty < 100):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out
#MAP
        if(penalty < 100):
            if(smooth):
                pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
            else:
                pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

            t =  tf.squeeze(thetas)                
                
            def ints(y): # called for y=1 and y=-1
                ky = iskequalsy(k,y)  # ky = 1 if k==y else -1
                if(debug):
                    print("ky",ky)
                out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
                if(debug):
                    print("intsy",out1)
                return out1                
                
            if(smooth):
                #smooth normalizer
                zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                               pcl,name="zy")
            else:
                #discrete normalizer
                zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                               pcl,name="zy")            
#new model

        if(penalty > 200):

#             z_y_new = [tf.reduce_prod(1 + 1/(-1*k*thetas) * (tf.exp(-1*k*thetas) - tf.exp(-1*k*thetas*user_alphas)) ), \
#                    tf.reduce_prod(1 + 1/(1*k*thetas) * (tf.exp(1*k*thetas) - tf.exp(1*k*thetas*user_alphas)) ) ]
            numbins = tf.constant(10, dtype=tf.float64)
            #TODO: 2 binned integration over the s values to allow stable gradients.
            sbin_widths = (1-user_alphas)/numbins
            sbins = tf.einsum('i,j->ij', sbin_widths, tf.cast(tf.range(0,numbins+1), dtype=tf.float64))+tf.expand_dims(user_alphas,1)
            sbins = tf.transpose(sbins)
            
            def pot(s):
                if(penalty%10 == 1):
                    return thetas*s-alphas
                elif(penalty%10 == 2):
                    return thetas*tf.clip_by_value(s-alphas,0,1)
                elif(penalty%10 == 3):
                    return thetas*s
                return 0
                                       
            # over the y-s it is okay to use a map function, but not over the LFs.
            z_y_new = tf.map_fn(lambda y: tf.reduce_prod(1 + tf.reduce_sum(tf.exp(-y*k*pot(sbins)), axis=0)), tf.constant([1,-1], dtype=tf.float64))
#             z_y_new = tf.reduce_sum(tf.exp(-1*k*thetas*sbins), axis=0)*sbin_widths/(1-user_alphas)
# *sbin_widths

            Z_new = tf.reduce_sum(z_y_new)
            print("new z ", Z_new)

#p theta without exp [y x i]
            # snap s_ to the nearest bin. 
            s_ = tf.round((s_-user_alphas)*numbins/(1-user_alphas))*sbin_widths + user_alphas
            logp_theta_new_t = tf.map_fn(lambda y:  tf.reduce_sum(-y*l*pot(s_), axis=1), tf.constant([1,-1], dtype=tf.float64))


            loss_new_t = tf.reduce_logsumexp(logp_theta_new_t, axis=0) - tf.log(Z_new)
            loss_new = tf.negative(tf.reduce_sum(loss_new_t))
        
#marginals as softmax

            marginals_new = tf.expand_dims(tf.nn.softmax(logp_theta_new_t, axis=0), 2)

        if(penalty < 100):
            
            if(debug):
                print("pout",pout)    
    #MAP
            t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                               name="t_pout")

            if(debug):
                print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)

        if(penalty < 100):
            marginals = tf.nn.softmax(t_pout,axis=0)
#         else:
#             marginals = marginals_new
        
            if(debug):
                print("zy",zy)
            logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
            if(debug):
                print("logz",logz)
    ##         tf.summary.scalar('logz', logz)
            lsp = tf.reduce_logsumexp(t_pout,axis=0)
            if(debug):
                print("lsp",lsp)
##         tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
    
        if(penalty == 20 or penalty > 100):
            a_t = tf.convert_to_tensor(a_pr, dtype=tf.float64)
            n_t = tf.convert_to_tensor(n_pr, dtype=tf.float64)
            r_t = tf.convert_to_tensor(r_pr, dtype=tf.float64)
        def get_loss_20():
#NEW MODEL

            # unnormalized_marginals [numY]
            unnormalized_marginals = [tf.reduce_prod(tf.exp(-thetas*LF_l) + 1), \
                                      tf.reduce_prod(tf.exp(thetas*LF_l) + 1)]
            # same for continuous, just use the new formulation with sbins for summation, no intergration
            Z = tf.reduce_sum(unnormalized_marginals)

            # unnormalized_marginals_tiled: [numLF, numY]
            # per-LF-marginals: [numLFs]
            per_LF_marginals = tf.gather(unnormalized_marginals, LF_label)

#             return per_LF_marginals
            # PÎ¸(kj=y): [numLFs]  (computing Eq 14)                       
            per_LF_agreement_prob = tf.exp(thetas)/(1 + tf.exp(thetas))*per_LF_marginals/Z

#             p_thetas =  tf.expand_dims(prod__,1) * exp_terms/(1+exp_terms) / Z
            p_thetas = per_LF_agreement_prob

            ptheta_terms = a_t * n_t * tf.log(p_thetas) + (1-a_t) * n_t * tf.log(1-p_thetas)
                                                   
#-------------------------------

            return tf.negative(tf.reduce_sum(ptheta_terms)) # add other loss component

        loss_recall = 0
        if(penalty > 100):
            numYs=2
            loss_new, per_lf_prob, marginals, per_lf_recall, pots = model_only.model(numYs, k, l, s, thetas, alphas, isdiscrete, user_a, penalty,alpha_max)
            loss_precision = model_only.precision_loss(a_t, n_t, per_lf_prob, penalty)
            if r_t is not None and (penalty//10) % 10 == 3:
                loss_recall = model_only.recall_loss(r_t, n_t, per_lf_recall, isdiscrete)

#adding ne      
           
        prec_loss = tf.constant(0)
    
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
            prec_loss = tf.constant(0)
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        

        elif(penalty == 4):
            print("precision penalty")
            prec_loss = tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g* prec_loss)
        
        elif(penalty == 41):
            print("precision penalty")
            prec_loss, xtra = sfp()
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g * prec_loss)
#                 + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)))
        
        elif(penalty == 42):
            print("precision penalty")
            temp_prloss, _ = sfpp()
            temp_uloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            r21 = tf.gradients(temp_uloss, thetas)
            r22 = tf.gradients(temp_prloss, thetas)
            xtra_tfg = tf.reduce_sum(tf.abs(r21[0][0]))/tf.reduce_sum(tf.abs(r22[0][0]))
#             gammaR = temp_uloss/temp_prloss
#             prec_loss, xtra = sfpp()
            prec_loss = tf.reduce_sum(temp_prloss)

            grad_ratio = 1

            xtra = [xtra_tfg]

            loss = temp_uloss \
                + (g * prec_loss)

        elif(penalty == 44):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_pp(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
                
                
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 15):
            print("equation 15 and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r15(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))



                    
        elif(penalty == 20):
            print("new model")
            prec_loss = get_loss_20()
            loss =  tf.negative(tf.reduce_sum(lsp  - logz  )) + prec_loss
                  
        elif(penalty >100 and penalty<111):
            sys.stdout = stdout_saved
            print("new model smooth ", penalty)
            # sys.stdout = open("trash", "w")            
            print("new model smooth ", penalty)
            loss = loss_new
#             loss, _ = smooth_new_prec()
            
                    
        elif(penalty >110):
            sys.stdout = stdout_saved
            print("new model smooth with constraints ", penalty)
            # sys.stdout = open("trash", "w")            
            print("new model smooth with constraints ", penalty)
            loss = loss_new + loss_precision + loss_recall
#             loss_un, loss_pr = smooth_new_prec()
#             loss = loss_un + loss_pr

        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            prec_loss = tf.constant(0)
       
        if(debug):
            print("loss",loss)

#         marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


        train_step = tf.train.AdamOptimizer(lr).minimize(loss)#, var_list=[thetas, alphas]) 


        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        
        dev_high = np.zeros(3)
        test_high = np.zeros(3)
        
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            flg = False
            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
#                 temprun = sess.run(next_element2)
#                 print('len next ',len(temprun), len(temprun[0]))
#                 print(temprun[0])
                tl = 0
                pll = 0
                unl = 0
                try:
                    it = 0
                    
                    grad_sum = 0.0
                    while True:
#                         print(en, it)

                        _,ls,ploss,lfProbs,t,al,p = sess.run([train_step,loss_new,loss_precision,per_lf_prob,thetas,alphas,pots])
                        if(penalty == 42):
                            un_loss = sess.run(temp_uloss)

                        tl = tl + ls
                        pll = pll + ploss
                        if(penalty == 42):
                            unl = unl + un_loss
                        it = it + 1
                        probs = np.exp(lfProbs)
                        if (np.amax(probs) > 1+1e-6):
                            print("Invalid probabilities=",probs)
                            
                        # print("various shapes=",p)
                        # print("Loss, prec_loss=", lfProbs)
                        # potsv, sv, lv, kv = sess.run([pots,s,l,k])
                        # print(t,al)
                        # print(sv,lv,kv)
                        # print(potsv)
#                         newptheta = sess.run(sbins)
#                         ztemp = sess.run(xyz)
                
#                         print(it*BATCH_SIZE)
                
                except tf.errors.OutOfRangeError:
                    pass
                opstring = ""
                print(en,"loss=",tl, " precision loss=",pll)
                opstring = opstring + str(en) + "," + str(tl) + ","
#                 print('temp__', temp__)
                
                print("ploss", pll)
                print("ratio", unl/pll)
#                 dloss = sess.run(dev_loss)
#                 print("dloss", dloss)
#                 print("expected ratio", xpp/(xpp+1))
                print("dev set")
                sess.run(dev_init_op)
##                 sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
#                 a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                a,t,m,pl,ddloss = sess.run([alphas,thetas,marginals,predict,loss])
##                 test_writer.add_summary(sm, en)
                print('init dev loss', ddloss)
                opstring = opstring + str(ddloss) + ","
                print("alphas=", a)
                print("thetas=",t)
                unique, counts = np.unique(pl, return_counts=True)
                print("Unique,counts of marginals", dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                dev_results = precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary")
                print('devresults ', dev_results)
                roc_dev = roc_auc_score(np.array(gold_labels_dev), np.array(m[1,:,0]))
                print('roc score', roc_dev)
                opstring = str(en) + ',' #do not print losses
                opstring = opstring + str(dev_results[2]) + "," + str(roc_dev) + ","
#                 if(dev_results[2]>dev_high[2]):
                dev_high = np.array(dev_results)
#                 dev_high = max(dev_high, dev_results[2])
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                test_results = precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary")
                print('testresults ', test_results)
                roc_test = roc_auc_score(np.array(gold_labels_test), np.array(m[1,:,0]))
                print('roc score', roc_test)
                opstring = opstring + str(test_results[2]) + "," + str(roc_test)
#                 if(test_results[2]>test_high[2]):
                test_high = np.array(test_results)
                print()
        
                sys.stdout = stdout_saved
                print(opstring)
                print(a, t)
# #                 newptheta = sess.run(marginals)
#                 print("new z ", ztemp)
#                 print("new z shape ", np.array(ztemp).shape)
#                 print('shape ',np.array(newptheta).shape)
                # sys.stdout = open("trash", "w")
                

    return pl, m, dev_high, opstring#test_high

In [ ]:
#added 2
r42d = []
r42t = []


a = 0.80

ua = np.repeat(a, len(LF_l))

stdout_saved = orig_stdout
# for rx in [1, len(LF_l), len(train_L_S)]:
# for i in np.linspace(1,1,1):
b = 32


print(discr)
for pen_i in [111]: # [101,102,103,104,105,106,111,112,113,114,115,116]:#[0, 4, 42]:
# for b in [32]:#[32,64,128,512,1024,2048,4096]:
    print(int(pen_i)//10 % 10)
    devmax = np.zeros(3)
    testmax = np.zeros(3)
    for gg in range(1):#[1]:#np.linspace(0,100,101):#, 10, 30, 50, 100]:
#         print("batch-size:",b, "gamma:",gg)
        print('============', pen_i)
        # sys.stdout = open('trash', 'w')
#        sys.stdout = stdout_saved
        _, _, _, ops = train2(1/len(train_L_S),200,batch_size = b, th = tf.truncated_normal_initializer(1,0,seed),\
                                    af = tf.truncated_normal_initializer(0,0.1,seed),\
                                    user_a = ua, \
                                    LF_acc = np.repeat(0.4, len(LF_l)) ,pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=pen_i, Gamma=gg, debug=False,\
                                    a_pr = np.repeat(0.750, len(LF_l)), \
                                           n_pr = np.repeat(600, len(LF_l)),\
                                     isdiscrete = dscr, alpha_max=np.repeat(0.9, len(LF_l)),
                             r_pr = np.repeat(0.750, len(LF_l)))
        sys.stdout = orig_stdout
# get_LF_acc(dev_L_S,gold_labels_dev)
#                 if(devmax_t[2]>devmax[2]):
#         devmax = np.array(devmax_t)
#                 if(testmax_t[2]>testmax[2]):
#         testmax = np.array(testmax_t)
#         r42d.append(devmax)
#         r42t.append(testmax)
#         print("\n\nfinal_result_line: ","thetas:",0,0.1,"batch size",b,"penalty",pen_i,)
##        sys.stdout = stdout_saved
#         print("\n\nfinal_result_line: ","thetas:",0,0.1,"batch size",b,"penalty",pen_i,)
# sys.stdout = open('opfile2', 'a')
# print('rate = 1/train_len')      
# sys.stdout = stdout_saved

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1
============ 111
Using binomial precision constraints
new model smooth with constraints  111
new model smooth with constraints  111
0 loss= 7340.07950314  precision loss= 4428301.47279
ploss 4428301.47279
ratio 0.0
dev set
init dev loss 6154.37324317
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 0.96954594  0.96954593  0.96954618  0.96954622  0.9695458   1.03058055
   0.96954582  1.0305806   1.03058062  1.03058009]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.778327034518

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.817024368804

0,0.428274428274,0.778327034518,0.49569707401,0.817024368804
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.041

8 loss= 6783.59926216  precision loss= 3606144.45372
ploss 3606144.45372
ratio 0.0
dev set
init dev loss 5169.29181931
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 0.8570675   0.85707586  0.85719471  0.85717696  0.8570798   1.19490607
   0.85706206  1.19485641  1.1948506   1.1948067 ]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.777482489292

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819183139062

8,0.428274428274,0.777482489292,0.49569707401,0.819183139062
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 0.8570675   0.85707586  0.85719471  0.85717696  0.8570798   1.19490607
   0.85706206  1.19485641  1.194850

17 loss= 7144.65141054  precision loss= 3490373.86422
ploss 3490373.86422
ratio 0.0
dev set
init dev loss 4998.56917672
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 1.02044685  1.02051835  1.0215066   1.02136415  1.02055179  1.43826064
   1.02040073  1.43813252  1.43811653  1.43799304]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.777436130008

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819194219149

17,0.428274428274,0.777436130008,0.49569707401,0.819194219149
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 1.02044685  1.02051835  1.0215066   1.02136415  1.02055179  1.43826064
   1.02040073  1.43813252  1.4381

26 loss= 7624.47724953  precision loss= 3376273.57649
ploss 3376273.57649
ratio 0.0
dev set
init dev loss 4837.39540074
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 1.20298881  1.20324616  1.2068137   1.2063338   1.20335666  1.71577213
   1.20281872  1.71563256  1.7156135   1.71546091]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.777430083144

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819194219149

26,0.428274428274,0.777430083144,0.49569707401,0.819194219149
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 1.20298881  1.20324616  1.2068137   1.2063338   1.20335666  1.71577213
   1.20281872  1.71563256  1.7156

35 loss= 8146.14851999  precision loss= 3284419.04057
ploss 3284419.04057
ratio 0.0
dev set
init dev loss 4707.56205304
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 1.38154429  1.38184651  1.38839808  1.38749071  1.38193233  1.98922845
   1.38133117  1.98906884  1.98904474  1.98884736]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.777430083144

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819194219149

35,0.428274428274,0.777430083144,0.49569707401,0.819194219149
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 1.38154429  1.38184651  1.38839808  1.38749071  1.38193233  1.98922845
   1.38133117  1.98906884  1.9890

44 loss= 8713.73271494  precision loss= 3209664.71611
ploss 3209664.71611
ratio 0.0
dev set
init dev loss 4601.92818284
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 1.56016664  1.56002825  1.56800665  1.56675488  1.55996585  2.26172843
   1.56051845  2.26154371  2.26151273  2.26125783]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.77744016125

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819321640148

44,0.428274428274,0.77744016125,0.49569707401,0.819321640148
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 1.56016664  1.56002825  1.56800665  1.56675488  1.55996585  2.26172843
   1.56051845  2.26154371  2.261512

53 loss= 9323.53884498  precision loss= 3148771.86326
ploss 3148771.86326
ratio 0.0
dev set
init dev loss 4515.91864439
alphas= [ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ]
thetas= [[ 1.73849901  1.73823875  1.74802727  1.74646266  1.73811849  2.53398121
   1.73913326  2.53376593  2.53372583  2.53340053]]
Unique,counts of marginals {0: 2522, 1: 292}
acc 0.902274342573
devresults  (0.35273972602739728, 0.544973544973545, 0.4282744282744283, None)
roc score 0.77744016125

test set
{0: 2339, 1: 363}
acc 0.89156180607
testresults  (0.39669421487603307, 0.66055045871559637, 0.49569707401032703, None)
roc score 0.819321640148

53,0.428274428274,0.77744016125,0.49569707401,0.819321640148
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 1.73849901  1.73823875  1.74802727  1.74646266  1.73811849  2.53398121
   1.73913326  2.53376593  2.533725

1,0.43006263048,0.762761400857,0.498327759197,0.818907060231
[ 0.11754463 -0.18578547  0.06448096  0.05554933  0.10637787  0.04133855
  0.22405651 -0.02316195  0.09187082  0.01032644] [[ 0.94514111  0.94514112  0.94514827  0.94514484  0.94514112  1.05462089
   0.9452137   1.05444394  1.05452913  1.05446836]]
   
   Cont: 111
   10,0.407002188184,0.746422776518,0.483636363636,0.80223891622
[ 0.11754463 -0.18578547  0.1191033   0.11100223  0.10637787  0.04133855
  0.26315504 -0.05675751  0.04734599 -0.02641954] [[ 0.90034616  0.90035037  0.89118076  0.89039836  0.90035211  1.12112825
   0.90581917  1.08926482  1.09932218  1.09215784]]
   
   Cont: 112
   9,0.296052631579,0.685242630385,0.375,0.737930092039
[ 0.11754463 -0.18578547  0.16593194  0.15628545  0.10637787  0.04133855
  0.32336718 -0.11055429 -0.00683302 -0.08423155] [[ 0.84517908  0.84518043  0.85065344  0.85047745  0.84518095  1.16100097
   0.85390063  1.16385916  1.16524347  1.16419562]]

    Cont: 112 with max_alpha = 0.7
    140,0.414587332054,0.780391030486,0.481300813008,0.830555924892
[ 1.11754463  0.81421453  1.00955965  1.00062431  1.10637787  1.04133855
  1.169206    1.03120785  1.14633471  1.0647233 ] [[ 0.2137293   0.22307502  4.16844262  3.91339975  0.23617798  1.17968475
   0.48896712  2.30967195  2.14310226 -0.46482738]]
   
   Discrete: 111
   10,0.428274428274,0.777492567397,0.49569707401,0.819183139062
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 0.82293943  0.82294084  0.82296135  0.82295826  0.82294152  1.18440805
   0.82293851  1.18439077  1.18438875  1.18437385]]
   
   Discrete: 112
   10,0.428274428274,0.777492567397,0.49569707401,0.819183139062
[ 0.11754463 -0.18578547  0.00955965  0.00062431  0.10637787  0.04133855
  0.169206    0.03120785  0.14633471  0.0647233 ] [[ 0.82293943  0.82294084  0.82296135  0.82295826  0.82294152  1.18440805
   0.82293851  1.18439077  1.18438875  1.18437385]]